<a href="https://colab.research.google.com/github/sushmithashenoy07/AgriMitra/blob/main/crop_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [46]:
crop_data_file = '../datasets/Crop_recommendation.csv'
market_data_file = '../datasets/cropmarketprice.csv'

In [47]:
crop_data = pd.read_csv(crop_data_file)
market_data = pd.read_csv(market_data_file)

In [48]:
print("Crop Data Columns:", crop_data.columns)
print("Market Data Columns:", market_data.columns)


Crop Data Columns: Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label'], dtype='object')
Market Data Columns: Index(['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade',
       'Arrival_Date', 'Min_x0020_Price', 'Max_x0020_Price',
       'Modal_x0020_Price'],
      dtype='object')


In [49]:
# Merge the two datasets on 'label' (from crop_data) and 'Commodity' (from market_data)
merged_data = pd.merge(crop_data, market_data, left_on='label', right_on='Commodity')


In [50]:
# Features from crop data: N, P, K, temperature, humidity, ph, rainfall
X = merged_data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]

# Target variable: Modal price from market data
y = merged_data['Modal_x0020_Price']

In [51]:
print("Crop Data Preview:")
crop_data.head()

Crop Data Preview:


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [52]:
print("Market Data Preview:")
market_data.head()

Market Data Preview:


,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_x0020_Price,Max_x0020_Price,Modal_x0020_Price
0,Andhra Pradesh,East Godavari,Karapa,Paddy(Dhan)(Common),1001,FAQ,19/09/2024,2183.0,2200.0,2183.0
1,Andhra Pradesh,East Godavari,Ravulapelem,Banana,Chakkarakeli(Red),Large,19/09/2024,2400.0,3200.0,2800.0
2,Andhra Pradesh,Guntur,Tenali,Lemon,Lemon,FAQ,19/09/2024,4200.0,7100.0,5800.0
3,Andhra Pradesh,Kurnool,Nandyal,Paddy(Dhan)(Common),Sona Mahsuri,FAQ,19/09/2024,2290.0,2290.0,2290.0
4,Assam,Barpeta,Barpeta Road,French Beans (Frasbean),French Beans (Frasbean),Local,19/09/2024,3800.0,4000.0,4000.0


In [53]:
crop_data['label'] = crop_data['label'].str.strip()
market_data['Commodity'] = market_data['Commodity'].str.strip()

In [54]:
# Merge the two datasets again
merged_data = pd.merge(crop_data, market_data, left_on='label', right_on='Commodity')

In [55]:
merged_data

,N,P,K,temperature,humidity,ph,rainfall,label,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_x0020_Price,Max_x0020_Price,Modal_x0020_Price


In [56]:
# Check if the merged data has rows
print(f"Merged Data Shape: {merged_data.shape}")
print("Merged Data Preview:")
print(merged_data.head())

Merged Data Shape: (0, 18)
Merged Data Preview:
Empty DataFrame
Columns: [N, P, K, temperature, humidity, ph, rainfall, label, State, District, Market, Commodity, Variety, Grade, Arrival_Date, Min_x0020_Price, Max_x0020_Price, Modal_x0020_Price]
Index: []


In [57]:
# Print unique crop names from both datasets
print("Unique crop names in crop data (label column):")
print(crop_data['label'].unique())

print("\nUnique commodities in market data (Commodity column):")
print(market_data['Commodity'].unique())


Unique crop names in crop data (label column):
['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']

Unique commodities in market data (Commodity column):
['Paddy(Dhan)(Common)' 'Banana' 'Lemon' 'French Beans (Frasbean)'
 'Bhindi(Ladies Finger)' 'Cabbage' 'Pumpkin' 'Capsicum' 'Onion' 'Potato'
 'Bottle gourd' 'Tomato' 'Soyabean' 'Coriander(Leaves)'
 'Green Gram (Moong)(Whole)' 'Cucumbar(Kheera)' 'Ginger(Green)'
 'Green Chilli' 'Papaya' 'Pomegranate' 'Ridgeguard(Tori)'
 'Bajra(Pearl Millet/Cumbu)' 'Castor Seed' 'Isabgul (Psyllium)' 'Soanf'
 'Suva (Dill Seed)' 'Maize' 'Cotton' 'Wheat' 'Arhar (Tur/Red Gram)(Whole)'
 'Bitter gourd' 'Brinjal' 'Corriander seed' 'Groundnut'
 'Sesamum(Sesame,Gingelly,Til)' 'Mustard' 'Black Gram (Urd Beans)(Whole)'
 'Millets' 'Methi Seeds' 'Bengal Gram(Gram)(Whole)' 'Cummin Seed(Jeera)'
 'K

In [58]:
crop_to_commodity_mapping = {
    'rice': 'Paddy(Dhan)(Common)',
    'maize': 'Maize',
    'chickpea': 'Kabuli Chana(Chickpeas-White)',
    'kidneybeans': 'Rajma',
    'pigeonpeas': 'Arhar (Tur/Red Gram)(Whole)',
    'mothbeans': 'Moth Beans',
    'mungbean': 'Green Gram (Moong)(Whole)',
    'blackgram': 'Black Gram (Urd Beans)(Whole)',
    'lentil': 'Lentil (Masur)(Whole)',
    'pomegranate': 'Pomegranate',
    'banana': 'Banana',
    'mango': 'Mango',
    'grapes': 'Grapes',
    'watermelon': 'Water Melon',
    'muskmelon': 'Karbuja(Musk Melon)',
    'apple': 'Apple',
    'orange': 'Orange',
    'papaya': 'Papaya',
    'coconut': 'Coconut',
    'cotton': 'Cotton',
    'jute': 'Jute',
    'coffee': 'Coffee',
    'pigeonpeas': 'Arhar (Tur/Red Gram)(Whole)',
    'mustard': 'Mustard',
    'soybean': 'Soyabean',
    'groundnut': 'Groundnut',
    'wheat': 'Wheat',
    'millets': 'Millets',
    'sugarcane': 'Sugarcane',
    'potato': 'Potato',
    'onion': 'Onion',
    'tomato': 'Tomato',
    'brinjal': 'Brinjal',
    'cabbage': 'Cabbage',
    'cauliflower': 'Cauliflower',
    'bhindi': 'Bhindi(Ladies Finger)',
    'pumpkin': 'Pumpkin',
    'bottle_gourd': 'Bottle gourd',
    'ridge_gourd': 'Ridgeguard(Tori)',
    'bitter_gourd': 'Bitter gourd',
    'chillies': 'Green Chilli',
    'garlic': 'Garlic',
    'ginger': 'Ginger(Green)',
    'turmeric': 'Turmeric',
    'spinach': 'Spinach',
    'coriander': 'Coriander(Leaves)',
    'mint': 'Mint(Pudina)',
    'beetroot': 'Beetroot',
    'carrot': 'Carrot',
    'peas': 'Green Peas',
    'guava': 'Guava',
    'pineapple': 'Pineapple',
    'lime': 'Lime',
    'mousambi': 'Mousambi(Sweet Lime)',
    'jackfruit': 'Jack Fruit',
    'cashew': 'Cashewnuts',
    'sapota': 'Chikoos(Sapota)',
    'fig': 'Fig(Anjura/Anjeer)',
    'amla': 'Amla(Nelli Kai)',
    'jamun': 'Jamun(Narale Hannu)',
    'custard_apple': 'Custard Apple (Sharifa)',
}

In [59]:
crop_data['label_mapped'] = crop_data['label'].map(crop_to_commodity_mapping)

In [60]:
# Check if there are any unmapped values
unmapped = crop_data[crop_data['label_mapped'].isna()]
print(f"Unmapped crops: {unmapped['label'].unique()}")

Unmapped crops: []


In [61]:
crop_data = crop_data.dropna(subset=['label_mapped'])

In [62]:
# Merge crop data with market data on the mapped labels
merged_data = pd.merge(crop_data, market_data, left_on='label_mapped', right_on='Commodity')

In [63]:
# Check the result of the merge
print(f"Merged Data Shape: {merged_data.shape}")
print("Merged Data Preview:")
merged_data.head()

Merged Data Shape: (130700, 19)
Merged Data Preview:


,N,P,K,temperature,humidity,ph,rainfall,label,label_mapped,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_x0020_Price,Max_x0020_Price,Modal_x0020_Price
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Paddy(Dhan)(Common),Andhra Pradesh,East Godavari,Karapa,Paddy(Dhan)(Common),1001,FAQ,19/09/2024,2183.0,2200.0,2183.0
1,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Paddy(Dhan)(Common),Andhra Pradesh,Kurnool,Nandyal,Paddy(Dhan)(Common),Sona Mahsuri,FAQ,19/09/2024,2290.0,2290.0,2290.0
2,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Paddy(Dhan)(Common),Chattisgarh,Dhamtari,Belargaon,Paddy(Dhan)(Common),I.R. 36,FAQ,19/09/2024,1900.0,1900.0,1900.0
3,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Paddy(Dhan)(Common),Chattisgarh,Dhamtari,Boraee,Paddy(Dhan)(Common),I.R. 36,FAQ,19/09/2024,1950.0,1950.0,1950.0
4,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Paddy(Dhan)(Common),Chattisgarh,Dhamtari,Gattasilli,Paddy(Dhan)(Common),I.R. 36,FAQ,19/09/2024,1900.0,1900.0,1900.0


In [64]:
# Save the merged data to a new CSV file
merged_data.to_csv('../datasets/data.csv', index=False)

In [65]:
# Feature selection for machine learning
# X = merged_data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]  # Features from crop data
X = merged_data[['temperature', 'humidity', 'ph', 'rainfall','State','District','Market','Grade','Variety']]  # Features from crop data
y = merged_data['Modal_x0020_Price']  # Target: Market price

In [66]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model Training

## Neural Network Model for Crop prediction

In [67]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [68]:
df = pd.read_csv("../datasets/data.csv")

In [69]:
df.shape

(130700, 19)

In [70]:
df.sample(2)

,N,P,K,temperature,humidity,ph,rainfall,label,label_mapped,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_x0020_Price,Max_x0020_Price,Modal_x0020_Price
16798,60,54,19,18.748267,62.498785,6.417820,70.234016,maize,Maize,Tamil Nadu,Erode,Thalavadi(Uzhavar Sandhai ),Maize,Deshi Red,Local,19/09/2024,2000.0,2200.0,2200.0
34558,118,88,52,28.650039,82.687525,5.843163,98.750844,banana,Banana,Chattisgarh,Rajnandgaon,Rajnandgaon,Banana,Banana - Ripe,Medium,19/09/2024,4000.0,5000.0,4500.0


In [71]:
input_cols = ['temperature', 'humidity', 'ph', 'rainfall', 'State', 'District', 'Market', 'Grade', 'Variety','N','P','K']
target_cols = ['label' , 'Modal_x0020_Price']

In [72]:
class CropPricePredictor(tf.keras.Model):
    def __init__(self, input_shape,output_shape):
        super(CropPricePredictor, self).__init__()
        
        self.dense1 = tf.keras.layers.Dense(256,input_shape=input_shape, activation='leaky_relu')
        self.dense2 = tf.keras.layers.Dense(256,activation="leaky_relu")

        self.dense3 = tf.keras.layers.Dense(512, activation='relu')
        self.dense4 = tf.keras.layers.Dense(512, activation='relu')
        
        self.dense5 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense6 = tf.keras.layers.Dense(1024, activation='relu')
        
        # crop layers
        self.output_crop_1 = tf.keras.layers.Dense(512,activation="relu")
        self.output_crop_2 = tf.keras.layers.Dense(256,activation="relu")
        self.output_crop_3 = tf.keras.layers.Dense(64,activation="relu")

        self.output_crop = tf.keras.layers.Dense(output_shape, activation='softmax', name='crop_output')
        
        # price layers
        self.output_price_1 = tf.keras.layers.Dense(512, activation='relu')
        self.output_price_2 = tf.keras.layers.Dense(512, activation='relu')
        self.output_price_3 = tf.keras.layers.Dense(128, activation='relu')

        self.output_price = tf.keras.layers.Dense(1, name='price_output',activation='linear')
        
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs):        
        x = self.dense1(inputs)
        x = self.dropout(self.dense2(x))

        x = self.dropout(self.dense3(x))
        x = self.dropout(self.dense4(x))

        x = self.dropout(self.dense5(x))
        x = self.dropout(self.dense6(x))

        # Predict crop
        crop = self.dropout(self.output_crop_1(x))
        crop = self.dropout(self.output_crop_2(crop))
        crop = self.dropout(self.output_crop_3(crop))

        crop_output = self.output_crop(crop)

        # Predict price
        price = self.dropout(self.output_crop_1(x))
        price = self.dropout(self.output_crop_2(price))
        price = self.dropout(self.output_crop_3(price))

        price_output = self.output_crop(price)

        return {'price_output': price_output, 'crop_output': crop_output}

In [85]:
model = CropPricePredictor(input_shape=(len(input_cols),),output_shape=df['label'].unique().shape[0])

In [86]:
model.build(input_shape=(None, len(input_cols)))

In [87]:
model.summary()

Model: "crop_price_predictor_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            multiple                  3328      
                                                                 
 dense_29 (Dense)            multiple                  65792     
                                                                 
 dense_30 (Dense)            multiple                  131584    
                                                                 
 dense_31 (Dense)            multiple                  262656    
                                                                 
 dense_32 (Dense)            multiple                  525312    
                                                                 
 dense_33 (Dense)            multiple                  1049600   
                                                                 
 dense_34 (Dense)            multiple       

In [88]:
df = df[target_cols + input_cols]

In [89]:
df.sample(3)

,label,Modal_x0020_Price,temperature,humidity,ph,rainfall,State,District,Market,Grade,Variety,N,P,K
23752,16,7100.0,23.128082,92.683284,6.630646,109.393016,20,12,35,3,63,15,11,38
51719,1,3200.0,28.311933,75.773638,6.165001,119.695765,20,59,112,3,11,111,79,53
8822,17,2203.0,23.359054,83.595123,5.333323,188.413665,19,89,148,0,47,65,37,40


In [90]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in ['State', 'District', 'Market', 'Grade', 'Variety','label']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col].astype(str))
    

In [91]:
df.sample(3)

,label,Modal_x0020_Price,temperature,humidity,ph,rainfall,State,District,Market,Grade,Variety,N,P,K
13883,18,3000.0,18.525108,69.027623,5.773455,88.102344,10,180,86,2,17,72,60,25
16603,18,3000.0,18.928519,72.800861,6.158860,82.341629,10,132,185,2,17,90,57,24
79646,0,8625.0,22.490951,91.702927,5.795986,124.391510,13,75,402,3,38,23,138,195


In [92]:
x = df[input_cols]
y_crop = tf.keras.utils.to_categorical(df['label'])
y_price = df['Modal_x0020_Price']

In [93]:
X_train, X_test, y_crop_train, y_crop_test, y_price_train, y_price_test = train_test_split(x, y_crop, y_price, test_size=0.3, random_state=42)

In [94]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [95]:
model.compile(optimizer='adam', 
              loss={'price_output': 'mse', 'crop_output': 'categorical_crossentropy'}, 
              metrics={'price_output': 'mse', 'crop_output': 'accuracy'})

In [96]:
model.fit(X_train, {'price_output': y_price_train, 'crop_output': y_crop_train},
                    epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
1144/1144 [==============================] - 13s 10ms/step - loss: 48822592.0000 - crop_output_loss: 0.4554 - price_output_loss: 48822592.0000 - crop_output_accuracy: 0.8656 - price_output_mse: 48822592.0000 - val_loss: 49146264.0000 - val_crop_output_loss: 0.0535 - val_price_output_loss: 49146264.0000 - val_crop_output_accuracy: 0.9821 - val_price_output_mse: 49146264.0000
Epoch 2/10
1144/1144 [==============================] - 11s 10ms/step - loss: 48822488.0000 - crop_output_loss: 0.1262 - price_output_loss: 48822488.0000 - crop_output_accuracy: 0.9664 - price_output_mse: 48822488.0000 - val_loss: 49146264.0000 - val_crop_output_loss: 0.0518 - val_price_output_loss: 49146264.0000 - val_crop_output_accuracy: 0.9843 - val_price_output_mse: 49146264.0000
Epoch 3/10
1144/1144 [==============================] - 15s 13ms/step - loss: 48822500.0000 - crop_output_loss: 0.1041 - price_output_loss: 48822500.0000 - crop_output_accuracy: 0.9746 - price_output_mse: 48822500.0000 - val

In [97]:
model.evaluate(X_test, {'price_output': y_price_test, 'crop_output': y_crop_test})

1226/1226 [==============================] - 6s 5ms/step - loss: 47467840.0000 - crop_output_loss: 0.0083 - price_output_loss: 47467840.0000 - crop_output_accuracy: 0.9968 - price_output_mse: 47467840.0000


[47467840.0, 0.008270742371678352, 47467840.0, 0.9967610239982605, 47467840.0]

In [ ]:
model.save('../models/crop_price_predictor-100', save_format='tf')

In [ ]:
# label	Modal_x0020_Price	temperature	    humidity	ph	        rainfall	State	District	Market	Grade	Variety	    N	    P	    K
# 0	    7560.0	            21.377847	    92.720437	5.573241	106.141702	20	    12	        35	    3	    23	        30	    122	    197

input_values = [21.377847,92.720437,5.573241,106.141702,20,12,35,3,23,30,122,197]
input_tensor = tf.constant([input_values], dtype=tf.float32)

model(input_tensor)

## Using separate models to predict

In [98]:

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [100]:
class CropPredictor(tf.keras.Model):
    def __init__(self, input_shape, output_shape):
        super(CropPredictor, self).__init__()
        
        self.dense1 = tf.keras.layers.Dense(256,input_shape=input_shape, activation='leaky_relu')
        self.dense2 = tf.keras.layers.Dense(256,activation="leaky_relu")

        self.dense3 = tf.keras.layers.Dense(512, activation='relu')
        self.dense4 = tf.keras.layers.Dense(512, activation='relu')
        
        self.dense5 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense6 = tf.keras.layers.Dense(1024, activation='relu')

        self.output_1 = tf.keras.layers.Dense(128,activation="relu")
        self.output_2 = tf.keras.layers.Dense(64,activation="relu")
        self.output_layer = tf.keras.layers.Dense(output_shape, activation='softmax')

        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, x):
        x = self.dense1(x)
        x = self.dropout(self.dense2(x))

        x = self.dropout(self.dense3(x))
        x = self.dropout(self.dense4(x))

        x = self.dropout(self.dense5(x))
        x = self.dropout(self.dense6(x))

        x = self.dropout(self.output_1(x))
        x = self.dropout(self.output_2(x))

        return self.output_layer(x)

In [101]:
class PricePredictor(tf.keras.Model):
    def __init__(self, input_shape):
        super(PricePredictor, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128,input_shape=input_shape, activation='leaky_relu')
        self.dense2 = tf.keras.layers.Dense(256,activation="relu")

        self.dense3 = tf.keras.layers.Dense(512, activation='relu')
        self.dense4 = tf.keras.layers.Dense(512, activation='relu')

        self.output_1 = tf.keras.layers.Dense(128,activation="relu")
        self.output_2 = tf.keras.layers.Dense(32,activation="relu")
        self.output_layer = tf.keras.layers.Dense(1, activation='linear')

        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, x):
        x = self.dense1(x)
        x = self.dropout(self.dense2(x))

        x = self.dropout(self.dense3(x))
        x = self.dropout(self.dense4(x))

        x = self.dropout(self.output_1(x))
        x = self.dropout(self.output_2(x))

        return self.output_layer(x)

In [102]:
input_cols = ['temperature', 'humidity', 'ph', 'rainfall', 'State', 'District', 'Market', 'Grade', 'Variety','N','P','K']
target_cols = ['label' , 'Modal_x0020_Price']

In [103]:
df = pd.read_csv("../datasets/data.csv")

df = df[target_cols + input_cols]
df.sample(2)

,label,Modal_x0020_Price,temperature,humidity,ph,rainfall,State,District,Market,Grade,Variety,N,P,K
99406,papaya,3000.0,34.280461,90.555616,6.825371,98.540477,Tamil Nadu,Thiruvannamalai,Chengam(Uzhavar Sandhai ),Local,Papaya,44,60,55
32842,banana,2100.0,25.866324,84.423793,6.079179,114.535750,Uttar Pradesh,Rampur,Vilaspur,Medium,Banana - Ripe,93,94,53


In [104]:
df.head()

,label,Modal_x0020_Price,temperature,humidity,ph,rainfall,State,District,Market,Grade,Variety,N,P,K
0,rice,2183.0,20.879744,82.002744,6.502985,202.935536,Andhra Pradesh,East Godavari,Karapa,FAQ,1001,90,42,43
1,rice,2290.0,20.879744,82.002744,6.502985,202.935536,Andhra Pradesh,Kurnool,Nandyal,FAQ,Sona Mahsuri,90,42,43
2,rice,1900.0,20.879744,82.002744,6.502985,202.935536,Chattisgarh,Dhamtari,Belargaon,FAQ,I.R. 36,90,42,43
3,rice,1950.0,20.879744,82.002744,6.502985,202.935536,Chattisgarh,Dhamtari,Boraee,FAQ,I.R. 36,90,42,43
4,rice,1900.0,20.879744,82.002744,6.502985,202.935536,Chattisgarh,Dhamtari,Gattasilli,FAQ,I.R. 36,90,42,43


In [105]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in ['State', 'District', 'Market', 'Grade', 'Variety','label']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col].astype(str))

In [106]:
import pickle

with open('../models/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [107]:
x = df[input_cols]
y_crop = tf.keras.utils.to_categorical(df['label'])
y_price = df['Modal_x0020_Price']

In [116]:
crop_model = CropPredictor(input_shape=(len(input_cols),), output_shape=df['label'].unique().shape[0])
price_model = PricePredictor(input_shape=(len(input_cols)+1,))

In [117]:
crop_model.build(input_shape=(None, len(input_cols)))
price_model.build(input_shape=(None, len(input_cols)+1))

In [118]:
crop_model.summary()

Model: "crop_predictor_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            multiple                  3328      
                                                                 
 dense_57 (Dense)            multiple                  65792     
                                                                 
 dense_58 (Dense)            multiple                  131584    
                                                                 
 dense_59 (Dense)            multiple                  262656    
                                                                 
 dense_60 (Dense)            multiple                  525312    
                                                                 
 dense_61 (Dense)            multiple                  1049600   
                                                                 
 dense_62 (Dense)            multiple             

In [119]:
price_model.summary()

Model: "price_predictor_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            multiple                  1792      
                                                                 
 dense_66 (Dense)            multiple                  33024     
                                                                 
 dense_67 (Dense)            multiple                  131584    
                                                                 
 dense_68 (Dense)            multiple                  262656    
                                                                 
 dense_69 (Dense)            multiple                  65664     
                                                                 
 dense_70 (Dense)            multiple                  4128      
                                                                 
 dense_71 (Dense)            multiple            

In [120]:
x_crop_train, x_crop_test, y_crop_train, y_crop_test = train_test_split(x, y_crop, test_size=0.3, random_state=42)
x_price_train, x_price_test, y_price_train, y_price_test = train_test_split(df[input_cols + ["label"]], y_price, test_size=0.3, random_state=42)

In [121]:
crop_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
price_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [122]:
early_stopping_crop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
early_stopping_price = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=5, restore_best_weights=True)

In [123]:
crop_model.fit(x_crop_train, y_crop_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping_crop])

Epoch 1/100
1144/1144 [==============================] - 9s 7ms/step - loss: 0.4558 - accuracy: 0.8672 - val_loss: 0.0389 - val_accuracy: 0.9878
Epoch 2/100
1144/1144 [==============================] - 8s 7ms/step - loss: 0.1175 - accuracy: 0.9695 - val_loss: 0.0311 - val_accuracy: 0.9895
Epoch 3/100
1144/1144 [==============================] - 7s 7ms/step - loss: 0.0830 - accuracy: 0.9796 - val_loss: 0.0316 - val_accuracy: 0.9883
Epoch 4/100
1144/1144 [==============================] - 8s 7ms/step - loss: 0.0915 - accuracy: 0.9789 - val_loss: 0.0892 - val_accuracy: 0.9914
Epoch 5/100
1144/1144 [==============================] - 8s 7ms/step - loss: 0.0821 - accuracy: 0.9816 - val_loss: 0.0207 - val_accuracy: 0.9931
Epoch 6/100
1144/1144 [==============================] - 8s 7ms/step - loss: 0.0720 - accuracy: 0.9837 - val_loss: 0.0182 - val_accuracy: 0.9934
Epoch 7/100
1144/1144 [==============================] - 8s 7ms/step - loss: 0.0931 - accuracy: 0.9818 - val_loss: 0.0573 - val_ac

In [ ]:
price_model.fit(x_price_train, y_price_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping_price])

In [124]:
crop_model.evaluate(x_crop_test, y_crop_test)
price_model.evaluate(x_price_test, y_price_test)

1226/1226 [==============================] - 4s 3ms/step - loss: 47409232.0000 - mse: 47409232.0000


[47409232.0, 47409232.0]

In [125]:
crop_model.save('../models/crop_predictor-100', save_format='tf')
price_model.save('../models/price_predictor-100', save_format='tf')

INFO:tensorflow:Assets written to: ../models/crop_predictor-100\assets
INFO:tensorflow:Assets written to: ../models/price_predictor-100\assets


In [126]:
input_arr = [20.879744, 82.002744, 6.502985, 202.935536, 0, 55, 193, 0, 0, 90, 42, 43]
output_val = crop_model.predict(tf.constant([input_arr], dtype=tf.float32))

import numpy as np

print(np.argmax(output_val))
print(output_val)

1/1 [==============================] - 0s 172ms/step
17
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 3.0651446e-14 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 7.6634859e-18
  7.2306229e-37 0.0000000e+00 1.0000000e+00 0.0000000e+00]]


In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = train_test_split(df[input_cols], df['Modal_x0020_Price'], test_size=0.3, random_state=42)

booster = RandomForestRegressor(n_estimators=100, random_state=42)
booster.fit(x_price_train, y_price_train)

y_pred = booster.predict(x_price_test)
mse = mean_squared_error(y_price_test, y_pred)
print(f"MSE: {mse}")

MSE: 1296.7259949530685


In [128]:
import pickle

with open('../models/price_predictor.pkl', 'wb') as f:
    pickle.dump(booster, f)